In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Tokenizer, GPTJForCausalLM, DataCollatorWithPadding
from datasets import load_dataset

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("hf-internal-testing/tiny-random-gptj")

hface_model = GPTJForCausalLM.from_pretrained("hf-internal-testing/tiny-random-gptj")

Some weights of the model checkpoint at hf-internal-testing/tiny-random-gptj were not used when initializing GPTJForCausalLM: ['h.1.attn.bias', 'h.1.attn.v_proj.weight', 'h.4.ln_1.bias', 'h.3.ln_1.weight', 'h.0.mlp.fc_out.weight', 'h.1.ln_1.weight', 'h.0.attn.q_proj.weight', 'h.2.mlp.fc_in.weight', 'h.3.attn.k_proj.weight', 'wte.weight', 'h.1.attn.q_proj.weight', 'h.0.attn.bias', 'ln_f.bias', 'h.2.attn.bias', 'h.4.mlp.fc_out.weight', 'h.4.attn.masked_bias', 'h.4.attn.k_proj.weight', 'h.0.ln_1.weight', 'h.3.attn.out_proj.weight', 'h.4.mlp.fc_out.bias', 'h.1.mlp.fc_out.bias', 'h.2.attn.masked_bias', 'h.3.attn.bias', 'h.1.attn.masked_bias', 'h.1.mlp.fc_in.weight', 'h.0.attn.v_proj.weight', 'h.1.mlp.fc_in.bias', 'h.0.mlp.fc_in.weight', 'h.0.attn.out_proj.weight', 'h.2.mlp.fc_out.bias', 'h.2.ln_1.bias', 'h.0.mlp.fc_in.bias', 'h.1.mlp.fc_out.weight', 'h.4.attn.q_proj.weight', 'score.weight', 'h.4.ln_1.weight', 'h.2.attn.q_proj.weight', 'h.3.attn.masked_bias', 'h.4.mlp.fc_in.weight', 'h.1.ln_

In [3]:
hface_model

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(1000, 32)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=32, out_features=32, bias=False)
          (v_proj): Linear(in_features=32, out_features=32, bias=False)
          (q_proj): Linear(in_features=32, out_features=32, bias=False)
          (out_proj): Linear(in_features=32, out_features=32, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=32, out_features=128, bias=True)
          (fc_out): Linear(in_features=128, out_features=32, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((32,)

In [4]:
# Ignore this for now
#small_dataset = {
#    "rand": {
#        "prompts": [
#            "he's bald, he's mauled, he's called grindlewald... or so I'm tald.",
#            "To win, you must move forward at all costs. There is no retreat. I get knocked down, but I get up again. You will never keep me down.",
#            "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam", 
#            "quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum",
#            "dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
#        ],
#        "answers": [
#            "But why does the pilgrim seek refuge?",
#            "Backoff, headstrong",
#            "Sed ut perspiciatis unde omnis iste natus error",
#            "sit voluptatem accusantium doloremque laudantium",
#            "totam rem aperiam, eaque ipsa quae ab illo inventore",
#        ]
#    },
#    "blicket_direct": {
#        "prompts": [
#            "You have two items. At least one is a blicket. If an item touches a machine and it turns on, the item is a blicket. You take one of the items and place it on a machine. The machine does not turn on. Is the item a blicket?",
#            "You have two items. At least one is a blicket. If an item touches a machine and it turns on, the item is a blicket. You take one of the items and place it on a machine. The machine turns on. Is the item a blicket?",
#            "You have two tools. At least one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fail to fix it. Is the tool you used magic?",
#            "You have two tools. At least one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fix it! Is the tool you used magic?",
#            "I see four eggs sitting on a table. The waiter told me that at least 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins wobbly. Is that egg hardboiled?",
#            "I see four eggs sitting on a table. The waiter told me that at least 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins evenly. Is that egg hardboiled?",
#        ],
#        "answers": [
#            "no, it is not",
#            "yes, it is",
#            "no, it is not",
#            "yes, it is",
#            "no, it is not",
#            "yes, it is",
#        ]
#    },
#    "blicket_implicit": {
#        "prompts": [
#            "You have two items. At least one is a blicket. If an item touches a machine and the machine turns on, the item is a blicket. You take one of the items and place it on a machine. The machine does not turn on. Is the other item a blicket?",
#            "You have two items. At least one is a blicket. If an item touches a machine and the machine turns on, the item is a blicket. You take one of the items and place it on a machine. The machine turns on. Is the other item a blicket?",
#            "You have two items. Only one is a blicket. If an item touches a machine and it turns on, the item is a blicket. You take one of the items and place it on a machine. The machine turns on. Is the other item a blicket?",
#            
#            "You have two tools. At least one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fail to fix it. Is the tool you did not use magic?",
#            "You have two tools. At least one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fix it! Is the tool that you did not use magic?",
#            "You have two tools. Only one of the tools is magic. If a tool is magic, it can fix anything. If a tool is not magic, it cannot fix anything. You take one of the tools and try to fix a washing machine. You fix it! Is the tool that you did not use magic?",
#            
#            "I see four eggs sitting on a table. The waiter told me that at least 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins wobbly. Are the other eggs all hardboiled?",
#            "I see four eggs sitting on a table. The waiter told me that at least 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins evenly. Are the other eggs all hardboiled?",
#            "I see four eggs sitting on a table. The waiter told me that at only 3 of the eggs are hard boiled. If an egg is hard boiled, it will spin evenly. If it is not, it will spin wobbly. I see the waiter take one of the eggs and spin it. The egg spins evenly. Are the other eggs all hardboiled?",
#        ],
#        "answers": [
#            "yes, it is",
#            "we cannot know the answer from the given information",
#            "no, it is not",
#            
#            "yes, it is",
#            "we cannot know the answer from the given information",
#            "no, it is not",
#            
#            "yes, they are",
#            "we cannot know the answer from the given information",
#            "no, they are not",
#        ]
#    }
#}
#
#string = small_dataset["rand"]["prompts"][0]
#string

In [5]:
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

## Data Manipulations

In [6]:
MAX_SEQ_LEN = 100
bsize = 16

In [7]:
dataset = load_dataset("glue", "mrpc", split="train")
dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [8]:
tokenizer

PreTrainedTokenizer(name_or_path='hf-internal-testing/tiny-random-gptj', vocab_size=1000, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [9]:
new_tokens = {
    "pad_token": "[PAD]",
    "cls_token": "[CLS]", # Using CLS token as compression token
}


In [10]:
num_added = tokenizer.add_special_tokens(new_tokens)

In [11]:
n,h = hface_model.transformer.wte.weight.shape
n

1000

In [12]:
hface_model.resize_token_embeddings(n+num_added)

Embedding(1002, 32)

In [13]:
s = "heyo bo"
x = s[:100] + "[CLS]"
x

'heyo bo[CLS]'

In [14]:
def encode(examples):
    sent1 = [s + tokenizer.cls_token for s in examples["sentence1"]]
    inpts = tokenizer(sent1,padding="max_length", max_length=MAX_SEQ_LEN, truncation=True, return_tensors="pt")
    sent2 = [s + tokenizer.eos_token for s in examples["sentence2"]]
    outs = tokenizer(sent2,padding="max_length", max_length=MAX_SEQ_LEN, truncation=True , return_tensors="pt")
    
    examples["label"] = torch.LongTensor(examples["label"])
    idx = examples["label"]==0
    
    outs["input_ids"][idx] = inpts["input_ids"][idx].clone()
    outs["attention_mask"][idx] = inpts["attention_mask"][idx].clone()
    outs["input_ids"][outs["input_ids"]==tokenizer.cls_token_id] = tokenizer.eos_token_id
    
    idx = (inpts["input_ids"]==tokenizer.cls_token_id).float().reshape(len(idx),-1).sum(-1)
    if idx.sum() != len(idx):
        i = torch.argmax((idx==0).float())
        inpts["input_ids"][i,-1] = tokenizer.cls_token_id
    idx = (outs["input_ids"]==tokenizer.eos_token_id).float().reshape(len(examples["label"]),-1).sum(-1)
    if idx.sum() != len(examples["label"]):
        i = torch.argmax((idx==0).float())
        outs["input_ids"][i,-1] = tokenizer.eos_token_id
    
    #s = (inpts["input_ids"]==tokenizer.cls_token_id).float().reshape(len(idx),-1).sum()
    #if s != len(inpts["input_ids"]):
    #    print("CLS token id:", tokenizer.cls_token_id)
    #    print("Sum:", s)
    #    print("Len:", len(inpts["input_ids"]))
    #    x = inpts["input_ids"]
    #    idx = (x==tokenizer.cls_token_id).float().reshape(x.shape[0],-1).sum(-1)
    #    idx = torch.argmax((idx==0).float())
    #    print("idx:", idx)
    #    print(sent1[idx])
    #    print(inpts["input_ids"][idx])
    #    print()
    #    print()
    
    return {
        "input_ids":        inpts["input_ids"],
        "attention_mask":   inpts["attention_mask"],
        "output_ids":       outs["input_ids"],
        "output_attn_mask": outs["attention_mask"],
        "labels":           examples["label"],
    }
    
dataset = dataset.map(encode, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [15]:
dataset[0].keys()

dict_keys(['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask', 'output_ids', 'output_attn_mask', 'labels'])

In [16]:
len(dataset["input_ids"][0])

100

In [17]:
len(dataset["output_ids"][0])

100

In [18]:
dataset = dataset.remove_columns(["sentence1", "sentence2", "idx", "label"])
dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'output_ids', 'output_attn_mask', 'labels'])

In [19]:
dataset.set_format(type="torch")
#dataset.set_format(type="torch")
dataloader = torch.utils.data.DataLoader(dataset, batch_size=bsize, shuffle=True)

In [22]:
data = next(iter(dataloader))
idx = data["labels"]==0
print(data["input_ids"][idx][0])
print(data["output_ids"][idx][0])

tensor([  41,   78,  212,  418,   84,  224,  422,  320,  214,  243,  204,  236,
          85,  216,   68,  288,  201,  763,   69,  281,   75,  204,  274,  326,
         221,   74,  302,  557,  872,  437,   73,  257,  249,  268,   83,  487,
          69,  218,  198,  254,  386,  201,  203,  288,  223,  331,  398,  254,
         617,  267,  272,  213,  342,  660,  242,  274,  322,  227,  293,  788,
         209, 1001, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000])
tensor([  41,   78,  212,  418,   84,  224,  422,  320,  214,  243,  204,  236,
          85,  216,   68,  288,  201,  763,   69,  281,   75,  204,  274,  326,
         221,   74,  302,  557,  872,  437,   73,  257,  249,  268,   83,  487,
          69,  218,  198,  254,  386,  201,  203,  288,  223,  331,  398,  254,
       

## Data Verification

In [23]:
data = next(iter(dataloader))
with torch.no_grad():
    inpts = {
        "input_ids": data["input_ids"],
        "attention_mask": data["attention_mask"],
    }
    outputs = hface_model.transformer(**inpts)
    #logits = outputs.logits
    #print(logits.shape)

In [24]:
outputs.keys()

odict_keys(['last_hidden_state', 'past_key_values'])

In [25]:
outputs["last_hidden_state"].shape

torch.Size([16, 100, 32])

In [26]:
len(outputs["past_key_values"]) # Number of layers

5

In [27]:
outputs["past_key_values"][0][0].shape # Keys of first layer (or values, I'm not sure which, but probably keys because of naming key_values order)

torch.Size([16, 4, 100, 8])

In [28]:
# model.transformer(inputs_embeds=outputs["last_hidden_state"], attention_mask=inpts["attention_mask"]) # Use this to argue embeddings instead of ids

In [29]:
data["labels"][:10]

tensor([0, 0, 0, 1, 1, 1, 0, 1, 0, 1])

Positive Sample

In [30]:
tokenizer.decode(data["input_ids"][0])

'With the economy sluggish, producers have had a tough time trying to raise prices. [CLS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [31]:
tokenizer.decode(data["output_ids"][0])

'With the economy sluggish, producers have had a tough time trying to raise prices.<|endoftext|> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'


Negative Sample

In [32]:
tokenizer.decode(data["input_ids"][1])

'Gore said while the Patriot Act made some needed changes, it has " turned out to be, on balance, a terrible mistake. " [CLS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [33]:
tokenizer.decode(data["output_ids"][1])

'Gore said while the Patriot Act made some needed changes, it has " turned out to be, on balance, a terrible mistake. "<|endoftext|> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

## Model

In [34]:
class SentenceAutoEncoder(torch.nn.Module):
    """
    Trains a new token type to compress a sentence into a single vector representation
    """
    def __init__(self, model, tokenizer, *args, **kwargs):
        """
        model: hugging face transformer model
        """
        super().__init__()
        self.hface_model = model
        self.tokenizer = tokenizer
        self.CLS_ID = tokenizer.cls_token_id
        self.CLS = tokenizer.cls_token
        self.EOS_ID = tokenizer.eos_token_id
        self.EOS = tokenizer.eos_token
        
    def forward(self, data):
        """
        data: dict
            "input_ids": LongTensor (B,S1)
                the token indices of the input sequence. The CLS token should be appended to the end of each sentence.
            "attention_mask": LongTensor (B,S1)
                attention mask for padding purposes. 0s mean padding.
            "output_ids": LongTensor (B,S2)
                the token indices of the target sequence. An EOS token should be appended to the end of each sentence
            "output_attn_mask": LongTensor (B,S2)
                attention mask for padding purposes. 0s mean padding.
        """
        
        model = self.hface_model
        inpt_embs = model.transformer.wte(data["input_ids"]).data
        idx = data["input_ids"]==self.CLS_ID
        inpt_embs[idx] = 0
        inpt_embs[idx] += model.transformer.wte.weight[self.CLS_ID]
        out_embs =  model.transformer.wte(data["output_ids"]).data
        
        
        fx = model.transformer(inputs_embeds=inpt_embs, attention_mask=data["attention_mask"])
        fx = fx["last_hidden_state"][idx][:,None]
        
        
        try:
            out_embs = torch.cat([fx,out_embs], dim=1)
            attn = torch.cat([torch.ones_like(data["output_attn_mask"][:,:1]), data["output_attn_mask"]], dim=1)
        except:
            print("Data")
            for k in data: print(k, data[k].shape)
            print("idx sum:", idx.float().sum())
            print("In Embds", inpt_embs.shape)
            print("Out Embds", out_embs.shape)
            print("FX", fx.shape)
            assert False
        
        preds = model(inputs_embeds=out_embs, attention_mask=attn).logits
        return preds
        
        
        
        
        

In [35]:
model = SentenceAutoEncoder(hface_model, tokenizer)

## Training

In [36]:
lr = 1e-4
l2 = 1e-3
n_epochs = 50

In [37]:
optimizer = torch.optim.Adam(model.hface_model.transformer.wte.parameters(), lr=lr, weight_decay=l2)
loss_fxn = torch.nn.CrossEntropyLoss()

In [38]:
model.cuda()
for epoch in range(n_epochs):
    print()
    print("Beginning Epoch", epoch)
    model.train()
    avg_loss = 0
    avg_acc = 0
    for i,data in enumerate(dataloader):
        data = {k: v.cuda() for k,v in data.items()}
        preds = model(data)
        preds = preds[:,:-1].reshape(-1, preds.shape[-1])
        labels = data["output_ids"].reshape(-1)
        idx = labels!=tokenizer.pad_token_id
        loss = loss_fxn(preds[idx], labels[idx])
        avg_loss += loss.item()
        argmax = torch.argmax(preds[idx])
        acc = (argmax==labels[idx]).float().mean()
        avg_acc += acc.item()
        if i%10==0: print("Loss", round(loss.item(), 5), "-- Acc:", round(acc.item(), 5))
    print("Avg Loss:", round(avg_loss, 5), "-- Avg Acc:", round(avg_acc, 5))
    print()
        
        


Beginning Epoch 0
Loss 6.91259 -- Acc: 0.0
Loss 6.91193 -- Acc: 0.0
Loss 6.92265 -- Acc: 0.0
Loss 6.92159 -- Acc: 0.0
Loss 6.92124 -- Acc: 0.0
Loss 6.91839 -- Acc: 0.0
Loss 6.92472 -- Acc: 0.0
Loss 6.92621 -- Acc: 0.0
Loss 6.92465 -- Acc: 0.0
Loss 6.91898 -- Acc: 0.0
Loss 6.91775 -- Acc: 0.0
Loss 6.92092 -- Acc: 0.0
Loss 6.91833 -- Acc: 0.0
Loss 6.92593 -- Acc: 0.0
Loss 6.91921 -- Acc: 0.0
Loss 6.92712 -- Acc: 0.0
Loss 6.92914 -- Acc: 0.0
Loss 6.91767 -- Acc: 0.0
Loss 6.9271 -- Acc: 0.0
Loss 6.91894 -- Acc: 0.0
Loss 6.92484 -- Acc: 0.0
Loss 6.92443 -- Acc: 0.0
Loss 6.91963 -- Acc: 0.0
Avg Loss: 1591.58429 -- Avg Acc: 0.0


Beginning Epoch 1
Loss 6.92404 -- Acc: 0.0
Loss 6.92239 -- Acc: 0.0
Loss 6.91871 -- Acc: 0.0
Loss 6.91928 -- Acc: 0.0
Loss 6.92657 -- Acc: 0.0
Loss 6.91783 -- Acc: 0.0
Loss 6.92411 -- Acc: 0.0
Loss 6.91212 -- Acc: 0.0
Loss 6.91373 -- Acc: 0.0
Loss 6.91901 -- Acc: 0.0
Loss 6.91034 -- Acc: 0.0
Loss 6.91262 -- Acc: 0.0
Loss 6.91773 -- Acc: 0.0
Loss 6.91504 -- Acc: 0.0


KeyboardInterrupt: 